Copyright **`(c)`** 2024 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free under certain conditions — see the [`license`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [17]:
import logging
from itertools import combinations
import pandas as pd
import numpy as np
from geopy.distance import geodesic
import networkx as nx
import random

from icecream import ic

logging.basicConfig(level=logging.DEBUG)

In [ ]:
CITIES = pd.read_csv('cities/italy.csv', header=None, names=['name', 'lat', 'lon'])
DIST_MATRIX = np.zeros((len(CITIES), len(CITIES)))
for c1, c2 in combinations(CITIES.itertuples(), 2):
    DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic(
        (c1.lat, c1.lon), (c2.lat, c2.lon)
    ).km
CITIES.head()

## Lab2 - TSP

https://www.wolframcloud.com/obj/giovanni.squillero/Published/Lab2-tsp.nb

In [19]:
def tsp_cost(tsp):
    assert tsp[0] == tsp[-1]
    assert set(tsp) == set(range(len(CITIES)))
    tot_cost = 0
    for c1, c2 in zip(tsp, tsp[1:]):
        tot_cost += DIST_MATRIX[c1, c2]
    return tot_cost

## First Greedy Algorithm

In [ ]:
visited = np.full(len(CITIES), False)
dist = DIST_MATRIX.copy()
city = 0
visited[city] = True
tsp = list()
tsp.append(int(city))
while not np.all(visited):
    dist[:, city] = np.inf
    closest = np.argmin(dist[city])
    logging.debug(
        f"step: {CITIES.at[city,'name']} -> {CITIES.at[closest,'name']} ({DIST_MATRIX[city,closest]:.2f}km)"
    )
    visited[closest] = True
    city = closest
    tsp.append(int(city))
logging.debug(
    f"step: {CITIES.at[tsp[-1],'name']} -> {CITIES.at[tsp[0],'name']} ({DIST_MATRIX[tsp[-1],tsp[0]]:.2f}km)"
)
tsp.append(tsp[0])


logging.info(f"result: Found a path of {len(tsp)-1} steps, total length {tsp_cost(tsp):.2f}km")

## Simulated annealing

In [ ]:
# Parametri di Simulated Annealing
initial_temp = 5000  # Aumentata la temperatura iniziale
cooling_rate = 0.999  # Rallenta il raffreddamento
min_temp = 1e-3  # Temperatura minima

def simulated_annealing(tsp, initial_temp, cooling_rate, min_temp):
    current_solution = tsp.copy()
    current_cost = tsp_cost(current_solution)
    best_solution = current_solution.copy()
    best_cost = current_cost
    temperature = initial_temp

    while temperature > min_temp:
        # Genera una nuova soluzione scambiando due città a caso
        new_solution = current_solution.copy()
        i, j = np.random.randint(1, len(CITIES) - 1, size=2)
        new_solution[i], new_solution[j] = new_solution[j], new_solution[i]

        # Calcola il costo della nuova soluzione
        new_cost = tsp_cost(new_solution)

        # Calcola la differenza di costo
        cost_diff = new_cost - current_cost

        # Decidi se accettare la nuova soluzione
        if cost_diff < 0 or np.random.rand() < np.exp(-cost_diff / temperature):
            current_solution = new_solution
            current_cost = new_cost
            if current_cost < best_cost:
                ic("Do I enter here?")
                best_solution = current_solution
                best_cost = current_cost

        # Raffredda la temperatura
        temperature *= cooling_rate

    return best_solution, best_cost

best_solution, best_cost = simulated_annealing(tsp, initial_temp, cooling_rate, min_temp)


logging.info(f"Simulated annealing optimized solution cost: {best_cost:.2f} km")

## Simulated annealing with 2-opt

In [ ]:
# Parametri di Simulated Annealing
initial_temp = 5000  # Aumentata la temperatura iniziale
cooling_rate = 0.999  # Rallenta il raffreddamento
min_temp = 1e-3  # Temperatura minima

def two_opt(tour):
    """Applica la mossa 2-opt per invertire una parte del percorso."""
    i, j = np.sort(np.random.randint(1, len(CITIES) - 1, size=2))
    new_tour = tour[:i] + tour[i:j+1][::-1] + tour[j+1:]
    return new_tour

def simulated_annealing(tsp, initial_temp, cooling_rate, min_temp):
    current_solution = tsp.copy()
    current_cost = tsp_cost(current_solution)
    best_solution = current_solution.copy()
    best_cost = current_cost
    temperature = initial_temp

    while temperature > min_temp:
        # Genera una nuova soluzione usando 2-opt
        new_solution = two_opt(current_solution)

        # Calcola il costo della nuova soluzione
        new_cost = tsp_cost(new_solution)

        # Calcola la differenza di costo
        cost_diff = new_cost - current_cost

        # Decidi se accettare la nuova soluzione
        if cost_diff < 0 or np.random.rand() < np.exp(-cost_diff / temperature):
            current_solution = new_solution
            current_cost = new_cost
            if current_cost < best_cost:
                best_solution = current_solution
                best_cost = current_cost

        # Raffredda la temperatura
        temperature *= cooling_rate

    return best_solution, best_cost

best_solution, best_cost = simulated_annealing(tsp, initial_temp, cooling_rate, min_temp)


logging.info(f"Simulated annealing optimized solution cost: {best_cost:.2f} km")

## Evolutionary algorithm, first version

In [ ]:
# Parametri dell'algoritmo
POPULATION_SIZE = 100
GENERATIONS = 500
MUTATION_RATE = 0.2
ELITE_SIZE = 10

# Funzione per creare una soluzione randomica valida (un tour)
def create_random_solution():
    solution = list(range(len(CITIES)))
    np.random.shuffle(solution)
    solution.append(solution[0])  # Chiudiamo il ciclo
    return solution

def create_initial_population(size=POPULATION_SIZE):
    population = [tsp]  # Aggiungi la soluzione greedy
    population.extend(create_random_solution() for _ in range(size - 1))
    return population

# Funzione di fitness (usiamo il costo totale del tour)
def fitness(solution):
    return -tsp_cost(solution)  # Fitness inverso al costo (più alto è meglio)

# Selezione con torneo (scegliamo il migliore tra un sottogruppo casuale)
def tournament_selection(population, k=5):
    selected = random.sample(population, k)
    selected.sort(key=fitness, reverse=True)
    return selected[0]

# Crossover di ordine (Order Crossover - OX)
def order_crossover(parent1, parent2):
    size = len(parent1) - 1  # Ignoriamo l'ultimo elemento per il crossover
    start, end = sorted(random.sample(range(size), 2))
    
    child = [None] * size
    child[start:end] = parent1[start:end]
    
    # Fill the remaining positions with genes from parent2
    p2_index = 0
    for i in range(size):
        if child[i] is None:
            while parent2[p2_index] in child:
                p2_index += 1
            child[i] = parent2[p2_index]
    
    child.append(child[0])  # Richiudiamo il ciclo
    return child

# Mutazione 2-opt
def mutate(solution, mutation_rate=MUTATION_RATE):
    if random.random() < mutation_rate:
        # Applichiamo 2-opt selezionando due posizioni a caso
        i, j = sorted(random.sample(range(1, len(solution) - 1), 2))
        solution[i:j] = reversed(solution[i:j])
    return solution

# Algoritmo evolutivo principale
def evolutionary_algorithm():
    population = create_initial_population()
    best_solution = None
    best_fitness = float('-inf')

    for generation in range(GENERATIONS):
        # Valutazione e selezione dell'élite
        population.sort(key=fitness, reverse=True)
        elite = population[:ELITE_SIZE]
        
        # Aggiorna il miglior individuo trovato
        if fitness(elite[0]) > best_fitness:
            best_solution = elite[0]
            best_fitness = fitness(best_solution)

        # Generazione di una nuova popolazione
        new_population = elite[:]  # Manteniamo l'élite
        while len(new_population) < POPULATION_SIZE:
            parent1 = tournament_selection(population)
            parent2 = tournament_selection(population)
            child = order_crossover(parent1, parent2)
            child = mutate(child)
            new_population.append(child)
        
        population = new_population  # La nuova generazione diventa la popolazione corrente
        
        # Stampa di avanzamento
        if generation % 50 == 0:
            print(f"Generation {generation}: Best tour length = {-best_fitness:.2f} km")

    return best_solution, -best_fitness  # Restituiamo il tour migliore trovato e la sua distanza

# Esecuzione dell'algoritmo evolutivo
best_tour, best_cost = evolutionary_algorithm()
print(f"Best tour found: {best_tour} with total length {best_cost:.2f} km")
